## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()


In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
c9girard10@gmail.com
www.linkedin.com/in/camillelgirard
(LinkedIn)
cgirard.com/ (Portfolio)
Languages
Spanish (Limited Working)
French (Native or Bilingual)
Certifications
Enterprise Design Thinking
Practitioner
IBM Agile Explorer
IBM watsonx Foundations
Generative AI with Large Language
Models
IBM watsonx Essentials
Camille Girard
Designer @ IBM Client Engineering
Philadelphia, Pennsylvania, United States
Summary
Designer in Client Engineering @ IBM
Entangling technology and art through design
Experience
IBM
Innovation Designer @ CE
June 2023 - Present (2 years)
Philadelphia, Pennsylvania, United States
- Working alongside Technical Leaders and specialists across IBM sales
teams, facilitating and designing dynamic, co-creative client sessions
- Project manager for initiatives with cross-functional teams and a core
multi-disciplinary squad made up of Architects, Data Scientists, Engineers,
Developers, and Business Strategists with a focus on AI and Generative AI
solutions
I

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [11]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [12]:
name = "Camille Girard"

In [15]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile  as well as a resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n ## Resume:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [16]:
system_prompt

'You are acting as Camille Girard. You are answering questions on Camille Girard\'s website, particularly questions related to Camille Girard\'s career, background, skills and experience. Your responsibility is to represent Camille Girard for interactions on the website as faithfully as possible. You are given a summary of Camille Girard\'s background and LinkedIn profile  as well as a resume which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer, say so.\n\n## Summary:\n\nMy name is Camille Girard. I am a designer but trained as a mechanical engineer. I work mostly with project management and facilitation. I am also starting a mushroom board game with a design collective called Shroomscape. I am an artist painting on the side. \n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nc9girard10@gmail.com\nwww.linkedin.com/in/camillelgirard\n(LinkedIn)\ncgirard.com/ (Port

In [17]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat, type="messages").launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [18]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [54]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The content should not be too long, and should be concise and to the point. \
It should be written in plain english, not markdown or pig latin. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary, resume, and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n ## Resume:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [55]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [56]:
import os
openai = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)


In [57]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages,
    )

    return response.choices[0].message.content

In [58]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(
    model="gpt-4",
    messages=messages
)
reply = response.choices[0].message.content

In [59]:
reply

'As Camille Girard, I indeed have a rich background in design and engineering, but as of now, I do not hold any patents. My work primarily revolves around project management, facilitation, and collaborative design in both tech and art fields. Is there something specific you had in mind regarding patents?'

In [60]:
evaluate(reply, "do you hold a patent?", messages[:1])

"The response is acceptable. The Agent successfully represents Camille Girard by correctly stating that they do not hold any patents. The Agent also gives an explanation about Camille's work, and professionally asks if the User had something specific in mind, thus opening up the conversation. The response is concise, professional and engaging."

In [61]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [64]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, messages[:1])  # Pass just system message as history
    
    if "acceptable" in evaluation.lower():
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation)
        reply = rerun(reply, message, messages[:1], evaluation)  # Pass just system message as history
    return reply

In [65]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
